# RNA-Seq Workflow by @furkanmtorun 
### [furkanmtorun@gmail.com](mailto:furkanmtorun@gmail.com)  | GitHub: [@furkanmtorun](https://github.com/furkanmtorun)  | [Google Scholar](https://scholar.google.com/citations?user=d5ZyOZ4AAAAJ) | [Personal Website](https://furkanmtorun.github.io/) 

### Libraries , packages and required functions

In [ ]:
# +--------------------------------------------------+
# Import required librarys & packages
# +--------------------------------------------------+
import glob2
import subprocess

# +--------------------------------------------------+
# Define folders and bin for tools
# +--------------------------------------------------+
fastq_folder, genome_folder, index_folder, bam_sam_folder, logs_folder, results_folder, \
FastQC_bin, STAR_bin, cufflinks_bin, bowtie_bin, TopHat_bin, R_bin = ["./files/fastq/", 
            "./files/genome/", "./files/index/", "./files/bam_sam/", "./files/logs/", "./files/results/", "./softs/FastQC/", 
            "./softs/STAR-2.7.3a/bin/Linux_x86_64/", "./softs/cufflinks-2.2.1.Linux_x86_64/", 
            "./softs/bowtie2-2.3.5.1-linux-x86_64/", "./softs/tophat-2.1.1.Linux_x86_64/", 
            "C:/Program Files/R/R-3.6.1/bin/i386/"]


# +--------------------------------------------------+
# Define files
# +--------------------------------------------------+
fasta_files = " ".join(glob2.glob(genome_folder + "*.fa*"))
gtf_files = " ".join(glob2.glob(genome_folder + "*.gtf*"))
fastq_files = " ".join(glob2.glob(fastq_folder + "*.fastq*"))


# +--------------------------------------------------+
# The function for messages
# +--------------------------------------------------+
def msg_output(text):
    dash = "-"*len(text)
    dash = "-"*70 if len(dash) > 70 else "-"*len(dash)
    msg_txt = "\n# +" + dash + "+\n> {}\n# +" + dash + "+\n"
    print(msg_txt.format(text))
    

# +--------------------------------------------------+
# Execute and track the shell commands
# +--------------------------------------------------+
def run_command(command):
    try:
        return subprocess.check_output(command, shell=True)
    except (Exception, TypeError):
        msg_output("! Error!: Your command was:\n\t" + command)

        
# +--------------------------------------------------+
# Execute and track the shell commands
# +--------------------------------------------------+
def confirmation_runCommand(command):
    msg_output("Your command is:\n\t" + command)
    qa = input("> Are you OK with that command? Type 'YES' or 'NO': ")
    if qa.upper() == "YES":
        output = run_command(command).decode("utf-8")
        msg_output(output)
    elif qa.upper() == "NO":
        print("! You can change the command and then, re-run the cell")
    else:
        print("! Just type YES or NO: Please, re-run the cell")

### Quality Control using FastQC
##### Website: https://www.bioinformatics.babraham.ac.uk/projects/fastqc/

In [ ]:
fastqc_command = "{}fastqc {} -f fastq -o {}".format(FastQC_bin, fastq_files, results_folder + "QC_reports")
confirmation_runCommand(fastqc_command)

### Adapter Trimming using cutadapt
##### Website: https://cutadapt.readthedocs.io/en/

In [ ]:
preprocessing_ans = input("> Are the adapter sequnces of your FASTQ files trimmed? 'YES' or 'NO' : ")

if preprocessing_ans.upper() == "YES":
    msg_output("! The process was terminaled because the adapter sequences already trimmed.") 
elif preprocessing_ans.upper() == "NO":
    adapter_seq = input("> Paste your adapter sequence: ")
    number_of_threads = input("> Number Of Threads: ")
    if True == number_of_threads.isdigit() == adapter_seq.isalpha():
        for fastq_file in fastq_files.split(" "):
            fastq_file_name = fastq_file.split("\\")[1]
            cutadapt_command = "cutadapt -a {} -j {} {} -o {}trimmed_{}"\
                                .format(adapter_seq, number_of_threads, fastq_file, fastq_folder, fastq_file_name)
            confirmation_runCommand(cutadapt_command)
    else:
        msg_output("! Check the number of threads or the adapter sequence you have typed!")
else:
    msg_output("! Type only 'YES' or 'NO'.")

### Curation of Genome Index using BowTie2
#### Website: http://bowtie-bio.sourceforge.net/bowtie2/index.shtml

In [ ]:
bowtie_base_name = input("Type a basename for the files (e.g.: speciesName): ")
number_of_threads = input("> Number Of Threads: ")
extra_option = input("> Type your extra options: \n Check manual from http://bowtie-bio.sourceforge.net/bowtie2/manual.shtml: \n")

if True == number_of_threads.isdigit():
    bowtie_build_command = "{}bowtie2-build --threads {} {} {} {}"\
                            .format(bowtie_bin, number_of_threads, fasta_files, index_folder + bowtie_base_name, extra_option)
    confirmation_runCommand(bowtie_build_command)
    # To check your index, use following command: bowtie2-inspect -s <base_name>
else:
    msg_output("! Check the number of threads you have typed!")

### [Alternative] : Curation of Genome Index using STAR
##### Website: https://github.com/alexdobin/STAR

In [ ]:
run_mode = "genomeGenerate"
number_of_threads = input("> Number Of Threads: ")
overhang_number = input("> Overhang (ideally: ReadLength - 1): ")
extra_option = input("> Paste your extra options: \nUse formal manual: https://raw.githubusercontent.com/alexdobin/STAR/921a50b1b4730a2c8b6bffc03b85081e9de3f777/doc/STARmanual.pdf \nExample: --limitSjdbInsertNsj 4000 --limitGenomeGenerateRAM 269860224 --genomeSAindexNbases 12\n")

if True == number_of_threads.isdigit() == overhang_number.isdigit():
    if len(glob2.glob(genome_folder+"*.fasta")) > 0:
        run_command("gzip {}".format(genome_folder + "*fasta")).decode("utf-8")
    indexing_command = "{}STAR --runThreadN {} --runMode {} --genomeDir {} --genomeFastaFiles {} --sjdbGTFfile {} --sjdbOverhang {} {}" \
                        .format(STAR_bin, number_of_threads, run_mode, index_folder, fasta_files, gtf_files, overhang_number, extra_option)
    confirmation_runCommand(indexing_command)
else:
    msg_output("! Check the number of threads and overhang number you have typed!")


### Mapping/Alignment using TopHat
##### Website: http://ccb.jhu.edu/software/tophat/index.shtml

In [ ]:
library_type = input("> Library type 'fr-unstranded', ' fr-firststrand' or 'fr-secondstrand' : ")
number_of_threads = input("> Number Of Threads: ")
extra_option = input("> Type your extra options: \n Check manual from http://ccb.jhu.edu/software/tophat/manual.shtml#toph: \n")
msg_output("Please note that it is highly recommended that a FASTA file with the sequence(s) the genome being indexed be present \n in the same directory with the Bowtie index files and having the name <genome_index_base>.fa. \nIf not present, TopHat will automatically rebuild this FASTA file from the Bowtie index files.")

if True == number_of_threads.isdigit():
    # TO-DO: Find an elegant way to handle that problem with regex!
    reading_before_names = []
    # To take only file names of FASTQ files
    for fastq_file in fastq_files.split(" "):
        reading_before_names.append(fastq_file.split("_")[0].split("\\")[1])
    for read_file in list(set(reading_before_names)):
        read_files_together = ",".join(glob2.glob(fastq_folder + read_file + "*"))
        tophat_command = "{}tophat2 -p {} -o {} --library-type {} -G {} {} {} {}"\
                        .format(TopHat_bin, number_of_threads, bam_sam_folder + read_file, library_type, 
                                gtf_files, index_folder + "bowtie_base_name", read_files_together, extra_option)
        confirmation_runCommand(tophat_command)
else:
    msg_output("! Check the number of threads you have typed!")

### [Alternative]: Mapping/Alignment using STAR
##### Website: https://github.com/alexdobin/STAR

In [ ]:
number_of_threads = input("> Number Of Threads: ")
extra_option = input("> Type your extra options: \nUse formal manual: https://github.com/alexdobin/STAR\nExample: --outSAMunmapped Within --outSAMattributes Standard\n")

if True == number_of_threads.isdigit():
    reading_before_names = []
    # To take only file names of FASTQ files
    for fastq_file in fastq_files.split(" "):
        reading_before_names.append(fastq_file.split("_")[0].split("\\")[1])
    for read_file in list(set(reading_before_names)):
        read_files_together = ",".join(glob2.glob(fastq_folder + read_file + "*"))
        star_command = "{}STAR --runThreadN {} --genomeDir {} --readFilesIn {} --outFileNamePrefix {} --readFilesCommand zcat --outSAMtype BAM SortedByCoordinate {}" \
                .format(STAR_bin, number_of_threads, index_folder, read_files_together, bam_sam_folder + read_file, extra_option)
        confirmation_runCommand(star_command)
else:
    msg_output("! Check the number of threads you have typed!")

### Index BAM files (.BAI) using samtools
##### Website: http://www.htslib.org/

In [ ]:
bam_files = glob2.glob(bam_sam_folder + "*.bam")
bai_files = [bam_file + ".bai" for bam_file in bam_files]
for i in range(len(bam_files)):
    bai_command = "samtools index {} {}".format(bam_files[i], bai_files[i])
    confirmation_runCommand(bai_command)

msg_output("! Your .BAI and .BAM files are stored in the 'bam_sam' folder.\n You can visualize them using IGV.")

### Counting reads using HTSeq
##### Website: https://htseq.readthedocs.io/en/latest/count.html

In [ ]:
mode = input("> Choose a mode from 'union', 'intersection-strict' or 'intersection-nonempty' :")
stranded = input("> Data is stranded? 'yes', 'reverse' or 'no' : ")
order = input("> How the input data has been sorted? 'name' or 'pos' : ")
id_attribute = input("> Choose an id attribute? e.g: 'gene_id'")
feature_type = input("> Feature type (3rd column in GFF file) to be used? e.g: 'exon' : ")
extra_option = input("> Paste your extra options: e.g: --additional-attr=gene_name : ")

for bam_file in bam_files:
    output_fn = results_folder + "counts/" + bam_file.split("\\")[-1] + "_HTSeq.txt"
    htseq_command = "htseq-count -f bam -m {} -s {} -r {} -i {} -t {} {} {} {} > {}"\
                    .format(mode, stranded, order, id_attribute, feature_type, extra_option, bam_file, gtf_files, output_fn)
    confirmation_runCommand(htseq_command)

### [Alternative]: Counting reads using featureCounts
##### Website: http://subread.sourceforge.net/

In [ ]:
number_of_threads = input("> Number Of Threads: ")
stranded = input("> Data is stranded? 'yes', 'reverse' or 'no' : ")
id_attribute = input("> Choose an id attribute? e.g: 'gene_id'")
feature_type = input("> Feature type (3rd column in GFF file) to be used? e.g: 'exon' : ")
extra_option = input("> Type your extra options: (Check manual from http://bioinf.wehi.edu.au/featureCounts/)\n ")

# featureCount strand information
strand_conversion = {"no" : 0, "yes" : 1, "reverse" : 2}
try:
    stranded = strand_conversion[xz.lower()]
except Exception:
    msg_output("! Data is stranded? Select one of those: 'yes', 'reverse' or 'no' .")

if True == number_of_threads.isdigit():
    for bam_file in bam_files:
        output_fn = results_folder + "counts/" + bam_file.split("\\")[-1] + "_featureCounts.txt"
        featureCounts_command = "featureCounts -T {} -t {} -g {} -s {} {} -a {} -o {} {}"\
                        .format(number_of_threads, feature_type, id_attribute, stranded, 
                                extra_option, gtf_files, output_fn, bam_file)        
        confirmation_runCommand(featureCounts_command)
else:
    msg_output("! Check the number of threads you have typed!")

### Creating meta data file for the files

In [ ]:
ms_presence = input("Do you have a meta data with a full of comma seperated values? : 'YES' or 'NO' : ")
counts_files = [temp_file.split("\\")[1].split("_")[0] for temp_file in glob2.glob(results_folder + "counts/*.txt")]

print(counts_files)
if ms_presence.upper() == "YES":
    msg_output("! Your meta data file must be in the 'results' folder as 'meta_data.csv'\n\twith a full of comma seperated values including 'id,condition' header.")
elif ms_presence.upper() == "NO":
    meta_data_table = {"id" : "condition"}
    for count_file in counts_files:
        condition = input("> What is the condition for {} such as 'Control' or 'XYZ_gene mutant' ?".format(count_file))
        meta_data_table[count_file] = condition
        with open(results_folder + "meta_data.csv", "w") as meta_data_file:
            for meta_data_key in meta_data_table:
                meta_data_file.write(meta_data_key + "," + meta_data_table[meta_data_key] + "\n")
    msg_output("! Your meta data file has been created in the 'results' folder.")
else:
    msg_output("! Please, type either 'YES' or 'NO'.")


### Differential Expression Analysis using DESeq2
##### Website: https://bioconductor.org/packages/DESeq2